In [2]:
# 1. For reading:

# a WFDBrecord object with signals == None.  

item1 = rdheader('headername') # Function belongs to the module

# a WFDBrecord object with signals != None unless it is a 0 channel file. 
item1 = rdsamp('recordname') # Function belongs to the module. 

# Module function
def rdsamp(recordname):
    item1 = rdheader(recordname)
    item1.signals = rddats(item1.filenames)
    return item1


# Module function
def rdheader(recordname):
    f=open()
    return WFDBrecord

# Module function
def rddats(filenames):
    signals = np.fromfile(r)
    return signals



# 2. For writing

# Should wrhea be a module function or an object method? Can it only target WFDBrecord objects?
# Yes, we don't want it to be able to target strings. That's just a bother, why not just use write()?
# In fact, we can actually make some convenient string list to file functions part of the module. 

# lines is just a list of strings
def lines2header(headername, lines):
    f = open(headername+'.hea','w')
    for l in lines:
        f.write("%s\n" % l)
    f.close()

    

wrsamp calls wrheader with sig. wrheader calls setfields, and checkfields. 


setfields() - fills in fields of a wfdbrecord object. Fields that it can fill in include essential fields, and dependencies.

It will NOT overwrite any user defined fields, aka fields != None. 



Perhaps easiest solution is to specify a default order of filling in fields. Need to separate by record and signal? Nope, just put the record ones before the signal ones!!!!

In [1]:
# setfield and checkfield should NOT be visible to the user.



# Write a wfdb record's header and dat files. 
# Method belongs to the WFDBrecord class. Because it ONLY works on the class. 
# Usesignals = 0 (none), 1 (physical), 2 (digital) - The signals to use to write the file. 
def wrsamp(self, recordname, usesignals=0): # This belongs in signals.py
    
    if self.nsig>0 and usesignals = 0:
        sys.exit("Must specify which signals to use to write the WFDB files."
                 "\nusesignals = 1 for physical (p_signals), or 2 for digital (d_signals).")
    
    # Perform field validity and cohesion checks, and write the header file
    self.wrheader(recordname, usesignals)
    
    # Perform signal cohesiveness checks, and write all the dat files associated with the record
    self.wrdats()

# Write a header file. Method belongs to the WFDBrecord class.   
def wrheader(self, recordname, usesignals = 1):
     # Get all the fields needed to write the header
    writefields = self.getwritefields()
    
    # Use the chosen signals fields to potentially set missing required derived fields
    setsignalfeatures(usesignals, writefields)
        
    # Check the validity of individual fields used to write the header 
    self.checkfields(writefields, setmissing = 1, fieldcheckorder = singlefieldspeclist) 
    
    # Check the compatibility of fields used to write the header
    self.checkfieldcohesion()
    
    # Write the header file
    self.wrheaderfile(recordname)

# Convert the fields into a list of strings, then write those to a file. Method belongs to WFDB class.     
def wrheaderfile(self, recordname)
   
    recordline = []
    signallines = []
    commentlines = []
    line2header(recordname, recordline+signallines+commentlines)
    
# Write a list of strings to a file. Method belongs to module!!! 
def lines2header(recordname, lines):
    f = open(recordname+'.hea','w')
    for l in lines:
        f.write("%s\n" % l)
    f.close()
    
# Write all the dat files for a record. 
# Method belongs to the WFDBrecord class. NOT a function of the module. Does check the signal.  
def wrdats(self): # Don't need record names. The object fields specify. 
    
    # Perform extra check of the signal itself in terms of its values for writing. 
    # Get the final digital values to write to the file. 
    digitalwritesigs = self.checkwritesignals()
    
    # Get a list of the dat files to be written and their formats
    datfiles, datchannels = orderedsetlist(self.filename)
    formats = orderedsetlist(self.fmt)[0]
    
    for i in range(0, len(datfiles)):
        # To do: Consider multi samples per frame writing. 
        wrdat(datfiles[i], formats[i], self.signals[:,datchannels[i][0]:(datchannels[i][-1]+1)])
    
# Write a dat file. Signals is digital. 
# Belongs to the class but does not require 'self' to be passed in. So this class function can only be called from
# the class, not an instance of the class (object)
def wrdat(filename, fmt, signals):
    f=open(filename,'wb')
    np.tofile(f, signals)
    

# Returns the unique elements in a list in the order that they appear. 
# Also returns the indices of the original list that correspond to each output element. 
# Used for listing dat files and fmts to write. 
# Belongs to module
def orderedsetlist(fulllist):
    uniquelist = []
    ori = []
    for i in fulllist:
        if i not in uniquelist:
            uniquelist.append(i)
    return uniquelist, channels

# Returns elements in a list without consecutive repeated values. Used to check dat files to write. 
# Belongs to module
def orderednoconseclist(fulllist):
    noconseclist = [fulllist[0]]
    if len(fulllist) == 1:
        return noconseclist
    for i in fulllist:
        if i!= noconseclist[-1]:
            noconseclist.append(i)
    return noconseclist


# Get the list of required fields needed to write a wfdb header file (including multi-segment). 
# Returns the default required fields, the user defined fields, and their dependencies.
# This does NOT return signals or segments. Those are stated to be checked by other functions.
# Method of WFDBbaserecord
def getwritefields(self, fieldspeclist):
    # Get required record specification fields
    writefields=getwritesubset(self, reversed(list(fieldspeclist[1].items())))
    
    # Single-segment record
    if fieldspeclist[0] == signalspecs:
        err = checkfield(self, 'nsig', setmissing = 0):
        if err:
            sys.exit(err)
            
        # If nsig>0, get required signal specification fields. DON'T include signals here. It is checked in wrsamp.
        if self.nsig>0:
            writefields=writefields+['physical']+getwritesubset(self, reversed(list(fieldspeclist[2].items())))

    # Multi-segment record
    else:
        # Get required segment specification fields. DON'T add segments here. it will be checked in wrmultiheader
        writefields=writefields+['seglen', 'segname'] 
    # Comments
    if self.comments !=None:
        reqfields.append('comments')
    return reqfields


# Helper to getwritefields. 
# Method of WFDBbaserecord
def getwritesubset(self, fieldspecs):
    reqfields=[]
    for f in fieldspecs:
        if f in reqfields:
            continue
        # If the field is default required or has been defined by the user...
        if f[1].write_req or self.f!=None:
            rf=f
            # Add the field and its recurrent dependencies
            while rf!=None:
                reqfields.append(rf)
                rf=fieldspeclist[1][rf].dependency
    return reqfields    


# Use the chosen signals fields to potentially set missing required derived fields
# Method of WFDBrecord
def setsignalfeatures(self, usesignals, writefields):
    
    # Physical signal
    if usesignals == 1:
        
        # First, check the signal
        errormsg = self.checkfield(self, p_signals) # remember this doesn't check against other fields
        if errormsg:
            sys.exit(errormsg)
        # Fields potentially set via p_signals: nsig, siglen, (fmt, gain, baseline), (initvalue, checksum)
        
        if self.nsig == None:
            self.nsig = p_signals.shape[1]
        if self.siglen == None:
            self.siglen = p_signals.shape[0]
        
        # fmt is necessary to obtain gain and baseline.
        if self.fmt == 'None':
            res = estres(self.signals)
            self.fmt = self.nsig*[wfdbfmt(max(res))]
        else:
            errormsg = self.checkfield('fmt', 0) # remember, setdefault is for setting default of fields which do not depend on other fields.
            if errormsg:
                sys.exit(errormsg)
            # Have to check here whether fmt matches with filename? 
            
        
        
        # Check if d_signals is not none before overwriting it. 
        
        # Check fmt, gain, and baseline. Because usesignals == 1, automatically calculate gain and baseline. 
        # Better check the filenames vs fmt here too? 
        
        
        # Put a warning message if gain or baseline had been set and get overwritten. 
        
        
        
       
        
        elif field == 'fmt':
        if not checkfield(self, 'signals', 0):
            res = estres(self.signals)
            self.fmt = self.nsig*[wfdbfmt(max(res))]
    # adcgain, and baseline are related and depend on fmt.  
    # Do not infer a gain or baseline if the signal is already digital.
    elif field == 'adcgain':
        if self.physical == 1:
            # Calculate and set the gain and baseline.  
            if not checkfield(self, 'signals', 0):
                [gain, baseline] = adcparams(self.signals, self.fmt)
                self.adcgain = gain
                self.baseline = baseline
    elif field == 'baseline':
        if self.physical == 1:
            # Calculate and set the gain and baseline.  
            if not checkfield(self, 'signals', 0):
                [gain, baseline] = adcparams(self.signals, self.fmt)
                self.adcgain = gain
                self.baseline = baseline

    # Digital signal
    elif usesignals == 2:

        # First, check the signal
        errormsg = self.checkfield(self, d_signals) # remember this doesn't check against other fields
        if errormsg:
            sys.exit(errormsg)
            
        # Fields potentially set via d_signals: nsig, siglen, (initvalue, checksum)
        if self.nsig == None:
            self.nsig = d_signals.shape[1]
        if self.siglen == None:
            self.siglen = d_signals.shape[0]
        if 'initvalue' in writefields and self.initvalue == None:
            self.initvalue = list(self.d_signals[0,:])
        if 'checksum' in writefields and self.checksum == None:
            self.checksum = calc_checksum(self.d_signals) 
        
    return



# Check each field in the list of fields. If setmissing is true, try to set the field if it == None. 
# The checking order is specified by fieldcheckorder. Hopefully via the checking order, there will be no circular dependencies
# Fieldcheckorder starts from record line and moves onto signal/segment line.
# Method of WFDBbaserecord
def checkfields(self, fieldstocheck, setmissing=0, fieldcheckorder = []):
    # Check fields in order because the validity of some may be dependent on others. 
    # In addition, the setvalue of some may depend on other fields

    for f in fieldcheckorder:
        if f in fieldstocheck:
            self.checkfield(f, setmissing)
                
                
# Check whether a field is valid. If the 'setfield' option is 1, the function will try to set a default if possible. 
# Checkfield calls setdefault if setdefault == 1. setfield may call checkfield (on another field) with setmissing = 0.
# Method of WFDBbaserecord
def checkfield(self, field, setdefault=0):
    
    # The main part of this function, the individual specific field checks, does not call this function recursively.
    # ie. No fields are checked against other fields in order to determine their final validity. 
    
    # Do note however that this method may call setfield before the specific field checks, which may
    # call this function on another field. In the case that setfields does not work, it just returns without 
    # triggering an error, and subsequently, this function returns an error of 'missing field' for the original 
    # field checked, rather than an error of the field checked in 'setfields' that failed the check. 
    
    # This is a function for checking the basic validity of individual fields rather than checking the final
    # signal compatibility of all fields. But then we need another function to check cohesion for wrhea only? 
    
    errormsg = None
    
    # If specified, and the field is missing, try to set a default. 
    # Note, this may do nothing depending on the field. 
    if setdefault and self.field==None:
        self.setdefault(field)
        
    # If the field is (still) missing, trigger an error
    if self.field == None:
        errormsg = "Missing field required: "+field
        return errormsg

    # RETURNS??? FOLLOWING 
    
    # Check the number of elements in fields that should be lists.
    # Extend scalars into lists. 
    if field in sigfieldspecs or field in segfieldspecs or field == 'comments':
        self.checkfieldsize(field)      
        
    # Check the type of the field (or of its elements) 
    self.checkfieldtype(field)
    
    # Individual specific field checks:
    
    
    if field == 'd_signals':
        # Convert 1d into 2d array
        if self.d_signals.ndim == 1:
            self.d_signals.shape = [-1,1]  
        # Check shape
        if self.d_signals.ndim != 2:
            errormsg = "signals must be a 2d numpy array"
            return errormsg
        # Check dtype
        if self.d_signals.dtype not in [np.dtype('int64'), np.dtype('int32'), np.dtype('int16'), np.dtype('int8')]:
            errormsg = 'd_signals must be a 2d numpy array with dtype == int64, int32, int16, or int8.'
            return errormsg     
        # Make sure there are no values out of bounds for the format? NAAAAH NOT HERE (no need to check for nans)
        for ch in range(0, np.shape(digitalsignal)[1]):
            fmt = self.fmt[ch]
            dmin, dmax = digi_bounds(fmt)
            if (min(digitalsignal[:,ch])<dmin) or (max(digitalsignal[:,ch])>dmax):
                sys.exit("Channel "+str(ch)+" must only contain values between "+str(dmin)+" and "+str(dmax)+"for fmt = "+str(fmt))
            if np.isnan(digitalsignal[:,ch]).any():
                sys.exit("The digital signal cannot contain any Nans." 
                         "\nNan values are represented by: "+str(digi_nan(fmt))+" for fmt = "str(fmt))
        
    if field =='p_signals':        
        # Convert 1d into 2d array
        if self.signals.ndim == 1:
            self.signals.shape = [-1,1]  
        # Check shape
        if self.signals.ndim != 2:
            errormsg = "signals must be a 2d numpy array"

    # Figure this out later. 
    #elif field == 'segment':
        
    # Record specification fields
    elif field == 'recordname':       
        # Allowed letters, digits, and underscores.
        if re.match('\w+', self.recordname).string != self.recordname:
            errormsg = 'recordname must only comprise of letters, digits, and underscores.'
    elif field == 'nseg':
        if self.nseg <=0:
            errormsg = 'nseg must be a positive integer'
    elif field == 'nsig':
        if self.nsig <=0:
            errormsg = 'nsig must be a positive integer'
    elif field == 'fs':
        if self.fs<=0:
            errormsg = 'fs must be a positive number'
    elif field == 'counterfreq':
        if self.counterfreq <=0:
            errormsg = 'counterfreq must be a positive number'
    elif field == 'basecounter':
        if self.basecounter <=0:
            errormsg = 'basecounter must be a positive number' 
    elif field == 'siglen':
        if self.siglen <=0:
            errormsg = 'siglen must be a positive integer'
    # fix this time date shit
    elif field == 'basetime':
        if type(self.basetime == str): # If it's a WFDBtimeofday object, it's fine. 
            if not WFDBtimeofday.parsetimestring(self.basetime):
                errormsg = 'invalid time string format.'
    elif field == 'basedate':
        if type(self.basedate == str): # If it's a WFDBdate object, it's fine. 
            if not WFDBdate.parsedatestring(self.basedate):
                errormsg = 'invalid date string format.'
    
    # Signal specification fields. Already ensured to be lists of correct size at this point. 
    # Loop will exit after first error found.
    elif field == 'filename':
        # Check for filename characters
        for f in self.filename:
            acceptedstring = re.match('[\w]+\.?[\w]+',f)
            if not acceptedstring or acceptedstring != f:
                errormsg = 'File names should only contain alphanumerics and an extension. eg. record_100.dat'
                return errormsg  
        # Check that dat files are grouped together 
        if orderedsetlist(self.filename)[0] != orderednoconseclist(self.filename):
            errormsg = 'filename error: all entries for signals that share a given file must be consecutive'
            return errormsg
        # Check that each dat file has the same specified fmt
        datfmts = {}
        for ch in range(0,self.nsig):
            if self.filename[ch] not in datfmts:
                datfmts[self.filename[ch]] = self.fmt[ch]
            else:
                if datfmts[self.filename[ch]] != self.fmt[ch]:
                    errormsg = 'Each filename (dat file) specified must have the same fmt'
                    return errormsg
    elif field == 'fmt':
        for f in self.fmt:
            if f not in datformats:
                errormsg = 'File formats must be valid WFDB dat formats: '+' , '.join(datformats)
                break        
    elif field == 'sampsperframe':
        for f in self.sampsperframe:
            if f < 1:
                errormsg = 'sampsperframe values must be positive integers'
                break 
            sys.exit('Sorry, I have not implemented multiple samples per frame into wrsamp yet')
    elif field == 'skew':
        for f in self.skew:
            if f < 1:
                errormsg = 'skew values must be positive integers'
                break
            sys.exit('Sorry, I have not implemented skew into wrsamp yet')
    elif field == 'byteoffset':
        for f in self.byteoffset:
            if f < 1:
                errormsg = 'byteoffset values must be positive integers'
                break 
    elif field == 'adcgain':
        for f in self.adcgain:
            if f <= 0:
                errormsg = 'gain values must be positive numbers'
                break 
    elif field == 'baseline':
        # Currently original WFDB library only has 4 bytes for baseline. 
        for f in self.baseline:
            if f < -2147483648 or f> 2147483648:
                errormsg = 'baseline values must be between -2147483648 (-2^31) and 2147483647 (2^31 -1)'
                break 
    # Should we have a list of accepted units? Maybe leave this for now... 
    #elif field == 'units':
    elif field == 'adcres':
        for f in self.adcres:
            if f < 1:
                errormsg = 'adcres values must be positive integers'
                break 
    #elif field == 'adczero':
    
    # Checksum and initvalue are NOT checked against signals here, rather it is done in the signals check. 
    # elif field == 'initvalue':
    elif field == 'checksum':
        # Cannot be beyond 16 bit range.
        for f in self.checksum:
            if f< -65535 or f>65535:
                errormsg = 'checksum value lies beyond possible range'
                break
    #elif field == 'blocksize': # No support for blocksize

    #elif field == 'signame':
        # Note that we don't enforce a particular set of signal names in this library.
        # We can do that in a conversiontools library. 
        
        # Check for channelname characters. What to allow/disallow?
        
        #or f in self.signame:
        #    acceptedstring = re.match('[\w]+',f)
        #    if not acceptedstring or acceptedstring != f:
        #        errormsg = 'signames should only contain alphanumerics (including underscores)'
        #        return errormsg 
        
    
    # Segment specification fields
    elif field == 'segname':
        for f in self.segname:
            acceptedstring = re.match('[\w]+',f)
            if not acceptedstring or acceptedstring != f:
                errormsg = 'Segment record names should only contain alphanumerics'
                break  
    elif field == 'seglen':
        for f in self.seglen:
            if f < 1:
                errormsg = 'seglen values must be positive integers'
                break 
                
    # Comment field
    elif field == 'comments':
        for f in self.comments:
            if f[0] == '#':
                print("Note: The comment strings do not need to begin with '#'. This library adds them automatically.")
    
    return errormsg


# The reason why this checkfield method returns an error message rather than directly triggering an error is because
# checkfield is called by checkfields on fields we need to have, but it is also called by setfield to check other
# fields needed to set default values. So in the first case, we will exit using the error message. In the second case
# we will ignore the error message and continue without being able to set the field. 


# Check a signal, segment, or comment field to make sure its length is appropriate.
# Scalars will be extended into lists. 
# Belongs to WFDBbaserecord class
def checkfieldlength(self, field):
    
    if field == 'comments'
        # Single string comment. Convert it into a list.
        if type(self.field)!= list:
            self.field = [self.field]
    else:
        if field in sigfieldspecs:
            desiredlength = self.nsig
        elif field in segfieldspecs:
            desiredlength = self.nseg
        else:
            sys.exit('This function should not have been called on this field. Debug!')
            
        # Extend scalars into lists
        if type(self.field) != list:
            self.field = [self.field]*desiredlength

        if len(self.field)!=desiredlength:
            sys.exit('Field: '+field+' must have length '+str(desiredlength))
        

# Check the data type of the specified field.
# Belongs to WFDBbaserecord class
def checkfieldtype(self, field):
    
    # signal, segment, and comment specification fields are lists. Check their elements. 
    
    if field in recfieldspecs:
        if type(self.field) not in recfieldspecs[field].allowedtypes:
            print('Error - Field: '+field+'must be one of the following types:', recfieldspecs[field].allowedtypes)
            sys.exit()
            
    elif field in sigfieldspecs:
        for f in self.field:
            if type(f) not in sigfieldspecs[field].allowedtypes:
                print('Error - Field: '+field+'must be one of the following types:', sigfieldspecs[field].allowedtypes)
                sys.exit()
    elif field in segfieldspecs:
        for f in self.field:
            if type(f) not in segfieldspecs[field].allowedtypes:
                print('Error - Field: '+field+'must be one of the following types:', segfieldspecs[field].allowedtypes)
                sys.exit()
    elif field in comfieldspecs:
        for f in self.field:
            if type(f) not in comfieldspecs[field].allowedtypes:
                print('Error - Field: '+field+'must be one of the following types:', comfieldspecs[field].allowedtypes)
                sys.exit()
        
    
# Checks the fields against each other 
def checkfieldcohesion(self):
    
    # Digital and physical check? Or just check digital? Probably just digital is needed since physical isn't used. 
    
    
    
datformats = ["80","212","16","24","32"]
    
    
# Return min and max digital values for each format type.
def digi_bounds(fmt):
    if fmt == '80':
        return (-127, 127)
    elif fmt == '212':
        return (-2047, 2047)
    elif fmt == '16':
        return (-32767, 32767)
    elif fmt == '24':
        return (-8388607, 8388607)
    elif fmt == '32':
        return (-2147483647, 2147483648)
    
# Return nan value for the format type (also accepts lists) 
def digi_nan(fmt):
    if type(fmt) == list:
        diginans = []
        for f in fmt:
            diginans.append(digi_nan(f))
        return diginans
        
    if fmt == '80':
        return -128
    elif fmt == '212':
        return -2048
    elif fmt == '16':
        return -32768
    elif fmt == '24':
        return -8388608
    elif fmt == '32':
        return -2147483648
    
# Returns the analogue to digital conversion for a wfdb format signal.
# The signals, fmt, gain, and baseline fields must all be valid.
# Does not change the signals field of the object.
def adc(self):
    
    # The digital nan values for each channel
    dnans = digi_nan(fmt) 
    
    d_signal = self.signals * self.gain + self.baseline
    
    for ch in range(0, np.shape(self.signals)[1]):
        # Nan values 
        nanlocs = np.isnan(self.signals[:,ch])
        if nanlocs.any():
            d_signal[nanlocs,ch] = dnans[ch]
    
    return d_signal

# Returns the digital to analogue conversion for a wfdb format signal
# The signals, fmt, gain, and baseline fields must all be valid.
# Does not change the signals field of the object.
def dac(self):
    
    # The digital nan values for each channel
    dnans = digi_nan(fmt) 
    
    # Get nan indices, indicated by minimum value. 
    nanlocs = self.signals == dnans
    
    p_signal = (self.signals - self.baseline)/self.gain
        
    p_signal[nanlocs] = np.nan
            
    return p_signal
    
# Compute appropriate gain and baseline parameters given a physical signal and the fmts 
# Already considered for multiple formats. 
def adcparams(signals, fmt):
         
    # digital - baseline / gain = physical          !! gain CANNOT BE 0 !!
    # physical * gain + baseline = digital

    gains = []
    baselines = []
    
    # min and max ignoring nans, unless whole channel is nan. Should suppress error message. 
    minvals = np.nanmin(signals, axis=0) 
    maxvals = np.nanmax(signals, axis=0)
    
    dnans = digi_nan(fmt)
    
    for ch in range(0, np.shape(signals)[1]):
        dmin, dmax = digi_bounds(fmt[ch])
        dnan = dnans[ch]
        
        pmin = minvals[ch]
        pmax = maxvals[ch]
        
        # map values using full digital range.
        
        # If the entire signal is nan, just put any. 
        if pmin == np.nan:
            gain = 1 
            baseline = 1
        # If the signal is just one value, store all values as digital 1. 
        elif pmin == pmax:
            if minval ==0:
                gain = 1
                baseline = 1
            else:
                gain = 1/minval # wait.. what if minval is 0... 
                baselin = 0 
        else:
            
            gain = (dmax-dmin) / (pmax - pmin)
            baseline = dmin - gain * pmin

        # What about roundoff error? Make sure values don't map to beyond range. 
        baseline = np.floor(baseline) # baseline.astype('int64')
        
        # WFDB library limits...     
        if abs(gain)>214748364 or abs(baseline)>2147483648:
            sys.exit('Chen, please fix this')
                
        gains.append(gain)
        baselines.append(baseline)     
    
    return (gains, baselines)
    
    
# Retrieve a copy of the wfdb signals field, either digital or physical. 
# Returned array will NOT be a reference to the same object.
def getsignals(self, physical = 1)
    
    if not self.signals:
        sys.exit('WFDBrecord has no signals')
    
    # Return the physical signals
    if physical:
        if self.isphysical:
            returnsig = self.signals.copy()
        else:
            returnsig = self.dac()
            
    # Return the digital signals
    else:
        if self.isphysical:
            returnsig = self.adc()
        else:
            returnsig = self.signals.copy()
    
    return returnsig
    
    
    
    

signalspecs = OrderedDict([('signal', WFDBfield([[np.ndarray], None, None, False])),
                          ('physical', WFDBfield([[bool], None, None, False]))])

# The segment field. A list of WFDBrecord objects
segmentspecs = OrderedDict([('segment', WFDBfield([[list], None, None, True]))])

# Record specification fields            
recfieldspecs = OrderedDict([('recordname', WFDBfield([[str], '', None, True])),
                         ('nseg', WFDBfield([[int], '/', 'recordname', True])), # Essential for multi but not present in single.
                         ('nsig', WFDBfield([[int], ' ', 'recordname', True])),
                         ('fs', WFDBfield([[int, float], ' ', 'nsig', True])),
                         ('counterfreq', WFDBfield([[int, float], '/', 'fs', False])),
                         ('basecounter', WFDBfield([[int, float], '(', 'counterfreq', False])),
                         ('siglen', WFDBfield([[int], ' ', 'fs', True])),
                         ('basetime', WFDBfield([[str], ' ', 'siglen', False])),
                         ('basedate', WFDBfield([[str], ' ', 'basetime', False]))])
# Signal specification fields. May be np.ndarrays or lists. Or scalars if nsig = 1???  
sigfieldspecs = OrderedDict([('filename', WFDBfield([[str], '', None, True])),
                         ('fmt', WFDBfield([[int, str], ' ', 'filename', True])),
                         ('sampsperframe', WFDBfield([[int], 'x', 'fmt', False])),
                         ('skew', WFDBfield([[int], ':', 'fmt', False])),
                         ('byteoffset', WFDBfield([[int], '+', 'fmt', False])),
                         ('adcgain', WFDBfield([[int], ' ', 'fmt', True])),
                         ('baseline', WFDBfield([[int], '(', 'adcgain', True])),
                         ('units', WFDBfield([[str], '/', 'adcgain', True])),
                         ('adcres', WFDBfield([[int], ' ', 'adcgain', False])),
                         ('adczero', WFDBfield([[int], ' ', 'adcres', False])),
                         ('initvalue', WFDBfield([[int], ' ', 'adczero', False])),
                         ('checksum', WFDBfield([[int], ' ', 'initvalue', False])),
                         ('blocksize', WFDBfield([[int], ' ', 'checksum', False])),
                         ('signame', WFDBfield([[str], ' ', 'blocksize', False]))])
    
# Segment specification fields
segfieldspecs = OrderedDict([('segname', WFDBfield([[str], '', None, True, 0])),
                         ('seglen', WFDBfield([[int], ' ', 'segname', True, 0]))])
# Comment field
comfieldspecs = OrderedDict([('comments', WFDBfield([[int], '', None, False, False]))])





    
# Set the field if possible. Otherwise do nothing and return (no error message).
# This method WILL overwrite fields already set. Key is to not call it on fields already set to avoid this. 
# Although for gain and baseline when physical == 1, it will overwrite anyway. 
# Not all fields have defaults, and some fields' defaults may rely on other fields being present and valid. 
# The key is that these 'default' values for each field are NOT to be dependent on other fields, hence the
# world 'default'. There are other methods which set fields based on other fields. 
def setdefault(self, field):
    
    # Record specification fields
    if field == 'nsig':
        if not checkfield(self, 'signals', 0):
            self.nsig = np.shape(self.signals)[1]
    elif field == 'siglen':
        if not checkfield(self, 'signals', 0):
            self.nsig = np.shape(self.signals)[1]
    elif field == 'basetime':
        self.basetime = '00:00:00'
        
    # Signal specification fields    
    # This method will only suggest single dat records
    elif field == 'filename':
        self.filename = self.nsig*[self.recordname+'.dat']
        
    #elif field == 'fmt':
    #    if not checkfield(self, 'signals', 0):
    #        res = estres(self.signals)
    #        self.fmt = self.nsig*[wfdbfmt(max(res))]
    # adcgain, and baseline are related and depend on fmt.  
    # Do not infer a gain or baseline if the signal is already digital.
    #elif field == 'adcgain':
    #    if self.physical == 1:
            # Calculate and set the gain and baseline.  
   #         if not checkfield(self, 'signals', 0):
   #             [gain, baseline] = adcparams(self.signals, self.fmt)
   #             self.adcgain = gain
   #             self.baseline = baseline
   # elif field == 'baseline':
    #    if self.physical == 1:
            # Calculate and set the gain and baseline.  
    #        if not checkfield(self, 'signals', 0):
    #            [gain, baseline] = adcparams(self.signals, self.fmt)
    #            self.adcgain = gain
    #            self.baseline = baseline
    elif field == 'adcres':
        if not checkfield(self, 'fmt', 0):
            self.adcres=wfdbfmtres(self.fmt)
    elif field == 'adczero':
        if not checkfield(self, 'nsig', 0):
            self.adczero = self.nsig*[0]
    #elif field == 'initvalue':   
    #    if not checkfield(self, 'signals', 0):
            # physical or digital? 
    #        self.initvalue = self.signals[0,:]
    #elif field == 'checksum':
    #    if not checkfield(self, 'signals', 0):
            # Physical or digital? 
    #        self.initvalue = calc_checksum(self.signals) # class method. WFDBrecord.calc_checksum?
    elif field == 'blocksize':
        self.blocksize = 0
        
    
    # To do for multirecord (or maybe it's better not in this function): elif field == 'seglen':
 

# Estimate the resolution of each signal in a multi-channel signal in bits. Maximum of 32 bits. 
reslevels = np.power(2, np.arange(0,33))
def estres(signals):
    # Estimate the number of steps as the range divided by the minimum increment. 
    
    nsig = np.size(signals)[1]
    # The estimated resolution in bits rounded up
    res = np.zeros(nsig)
    
    for ch in range(0, nsig):
        sortedsig = np.sort(signals[:,ch])
        min_inc = min(np.diff(sortedsig))
        
        if min_inc == 0:
            # Case where signal is flat. Resolution is 0.  
            continue
        else:
            nlevels = 1 + (sortedsig[-1]-sortedsig[0])/min_inc
            if nlevels>=reslevels[-1]:
                res[ch] = 32
            else:
                res[ch] = np.where(reslevels>nlevels)[0][0]
            
    return res
    

# Return the most suitable wfdb format to use given a signal resolution. Limited in output options. 
def wfdbfmt(res):
    if res<=8:
        return '80'
    elif res<=12:
        return '212'
    elif res<=16:
        return '16'
    elif res<=24:
        return '24'
    else:
        return '32'

# Return the resolution of the WFDB format. Returns a list if the input has multiple items. 
def wfdbfmtres(fmt):
    
    if type(fmt) in [list, np.ndarray]:
        numel = np.shape(fmt)[0]
        res = np.zeros(numel)
        for i in range(0, numel):
            res[i] = wfdbfmtres(fmt[i])
        return res
    
    if fmt in ['8', '80']:
        return 8
    elif fmt in ['310', '311']:
        return 10
    elif fmt == '212':
        return 12
    elif fmt in ['16', '61']:
        return 16
    elif fmt = '24':
        return 24
    elif fmt = '32':
        return 32
    else:
        sys.exit('Invalid WFDB format.')
    
# Calculate the checksums of a multi-channel digital signal
# Method of WFDBrecord. Class method. 
def calc_checksum(signals):
    return list(np.sum(signals, 0) % 65536)


class WFDBtimeofday():
    
    def __init__(self, timestring):
        hours, minutes, seconds = parsetimestring(timestring)
            
        self.hours = hours
        self.minutes = minutes
        self.seconds = seconds
    
    # HH:MM:SS.sss format
    def parsetimestring(timestring):
        hours, minutes, seconds = re.findall("(?P<hours>\d{1,2}):(?P<minutes>\d{1,2}):(?P<seconds>\d{1,2}.?\d*)", timestring)[0]
    
        if not hours or not minutes or not seconds:
            sys.exit("Invalid time string: "+timestring". Acceptable format is: 'Hours:Minutes:Seconds'")
        if minutes>59:
            sys.exit('minutes must be < 60')            
        if seconds>59:
            sys.exit('seconds must be < 60')
        if hours <23:
            sys.exit('hours must be < 24')
        
        return (hours, minutes, seconds)
    
    
class WFDBdate():
    
    def __init__(self, datestring):
        day, month, year = WFDBdate.parsedatestring(datestring) 
        
        self.day = day
        self.month = month
        self.year = year
    
    # DD/MM/YYYY format    
    def parsedatestring(datestring):
        day, month, year = re.findall(r"(?P<day>\d{2})/(?P<month>\d{2})/(?P<year>\d{4})", datestring)[0]
        
        if not day or not month or not year:
            sys.exit("Invalid date string. Acceptable format is: 'DD/MM/YYYY'")
    
    

SyntaxError: invalid syntax (<ipython-input-1-cdf254654092>, line 44)